COMP5318 - Machine Learning and Data Mining: Assignment 1

In [74]:
import pandas as pd
import os
print(os.listdir("./Input/train"))
pd.set_option('display.max_columns', 10)

from IPython.display import set_matplotlib_formats, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import graphviz

%matplotlib inline

['train.csv']


In [75]:
# train.csv including feature and label using for training model.
data_train_df = pd.read_csv('./Input/train/train.csv') 

In [76]:
# Selecting input feature
data_train_feature = data_train_df.loc[:, "v1":"v784"].to_numpy()

# Selecting output lable 
data_train_label = data_train_df.label.to_numpy()

In [77]:
# import matplotlib.pyplot as plt
# data_train_feature = data_train_feature.reshape((data_train_feature.shape[0], 28, 28))
# plt.imshow(data_train_feature[0], cmap=plt.get_cmap('gray'))
# plt.title("class " + str(data_train_label[0]) + ": Pullover" )
# plt.show()

DATA PRE-PROCESSING FOR TRAINING DATA

In [78]:
pd.DataFrame(data_train_feature)

,0,1,2,3,4,...,779,780,781,782,783
0,0,0,0,0,1,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,1,0,...,61,0,0,0,0
4,0,0,0,0,0,...,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
29995,0,0,0,0,0,...,0,0,0,0,0
29996,0,0,0,0,0,...,0,0,0,0,0
29997,0,0,0,0,0,...,0,0,0,0,0
29998,0,0,0,0,0,...,0,0,0,0,0


In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    data_train_feature, data_train_label, random_state=0)

In [80]:
# Data Normalisation
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm  = scaler.transform(X_test)

pd.DataFrame(X_train_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.709804,0.156,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22495,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
22496,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
22497,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
22498,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0


In [81]:
pca = PCA(n_components=0.95).fit(X_train_norm)

X_train_pca = pca.transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)

pd.DataFrame(X_train_pca)

,0,1,2,3,4,...,450,451,452,453,454
0,-0.394598,-5.686051,0.913325,-1.125150,-1.570830,...,-0.089589,0.006516,0.033483,0.034051,0.034265
1,1.143164,-5.712517,1.264873,-1.803338,-1.362444,...,0.051334,0.058825,0.004493,-0.040664,0.013396
2,-6.141011,-0.336818,-0.597506,2.178537,-0.415588,...,0.112473,-0.030149,0.122586,0.135854,0.008785
3,-5.601226,2.431388,-0.185770,-2.443250,1.362475,...,-0.068558,-0.002467,0.030387,-0.180358,-0.074008
4,8.367414,3.090589,-2.711405,-0.902178,-0.412654,...,-0.101653,0.008830,-0.040168,0.013998,-0.064944
...,...,...,...,...,...,...,...,...,...,...,...
22495,0.867325,-4.352062,1.205116,-0.741638,-1.467599,...,-0.007702,-0.022681,0.002073,0.080159,-0.014667
22496,-6.117047,0.943566,-1.035591,-0.649070,0.617476,...,0.027286,0.017522,-0.006930,0.050267,0.013697
22497,-1.023785,-1.232009,-2.922173,1.696158,0.814303,...,0.055475,-0.050771,-0.069324,-0.025597,-0.057325
22498,0.604955,-6.065121,1.629911,-2.197434,-2.092257,...,0.082589,-0.022702,-0.046521,-0.042347,-0.040474


DATA PRE-PROCESSING FOR TESTING DATA

In [82]:
# test_input.csv includes 5000 samples used for label prediction. Test samples do not have labels.
data_test_df = pd.read_csv('./Input/test/test_input.csv', index_col = 0)

In [83]:
# Data Normalisation
output_test_norm = scaler.transform(data_test_df.to_numpy())

pd.DataFrame(output_test_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
2,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
3,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4996,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4997,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4998,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0


In [84]:
# Data Standardisation
data_test_df = StandardScaler().fit_transform(data_test_df)

In [85]:
# Dimension Reduction
output_test_pca = pca.transform(output_test_norm)

pd.DataFrame(output_test_pca)

,0,1,2,3,4,...,450,451,452,453,454
0,1.978747,-5.457940,1.946917,-1.959688,-1.010971,...,0.043663,0.015463,-0.057184,-0.045916,0.022597
1,1.923139,-5.680224,1.295473,-1.416505,-0.594287,...,0.023877,-0.038619,0.004406,0.000270,0.038203
2,0.472941,-5.856364,1.597844,-2.046583,-2.592412,...,-0.035371,0.021712,-0.005810,-0.035484,-0.034361
3,-0.375038,-3.365100,-0.763618,1.793366,0.032026,...,0.038208,0.040231,-0.052936,-0.032351,0.011913
4,-1.540833,-2.601513,-0.900526,0.412424,0.679410,...,0.051673,-0.122417,-0.018422,0.060033,-0.041456
...,...,...,...,...,...,...,...,...,...,...,...
4995,2.739363,-3.216671,0.817188,0.588171,-0.349807,...,0.102551,0.037557,-0.021889,-0.021035,-0.096540
4996,2.462737,5.937648,-0.364203,1.201229,3.887794,...,-0.140343,-0.057482,-0.129762,-0.032134,0.073361
4997,-5.694177,-0.599542,-1.165802,3.353452,-0.944159,...,0.192050,0.007689,-0.076368,-0.020860,-0.110781
4998,7.058567,1.491336,0.489352,1.188750,0.217547,...,-0.049835,-0.103863,-0.071675,0.135833,0.066556


KNN

In [13]:
print("X_train_pca shape:", X_train_pca.shape)
print("y_train shape:", y_train.shape)

X_train_pca shape: (22500, 20)
y_train shape: (22500,)


In [14]:
print("X_test_pca shape:", X_test_pca.shape)
print("y_test shape:", y_test.shape)

X_test_pca shape: (7500, 20)
y_test shape: (7500,)


In [18]:
param_grid = {'n_neighbors': [1, 3, 5, 11, 15], 'p': [1, 2]}
print("Parameter grid:\n{}".format(param_grid))

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)

grid_search.fit(X_train_pca, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test_pca, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

best_n = grid_search.best_params_['n_neighbors']
best_p = grid_search.best_params_['p']
knn = KNeighborsClassifier(n_neighbors=best_n, p=best_p)
knn.fit(X_train_pca, y_train)

Parameter grid:
{'n_neighbors': [1, 3, 5, 11, 15], 'p': [1, 2]}


KeyboardInterrupt: 

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# KNN Predictions
predictions = []

for i in output_test_pca:
    prediction = knn.predict([list(i)])
    predictions.append(prediction[0])

predictions

In [ ]:
output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('test_output.csv', sep=",", float_format='%d', index_label="id")

LOGISTIC REGRESSION

In [34]:
logreg = LogisticRegression(max_iter = 5000)
logreg.fit(X_train_pca, y_train)
print("Accuracy on trainig set:", logreg.score(X_train_pca, y_train)*100)
print("Accuracy on test set:", logreg.score(X_test_pca, y_test)*100)

Accuracy on trainig set: 84.00888888888889
Accuracy on test set: 83.09333333333333


Naive Bayes

In [422]:
nb = GaussianNB()
nb.fit(X_train_pca, y_train)
y_pred = nb.predict(X_test_pca)
print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy on test set: 0.669


In [ ]:
# Parameter Tuning 
param_grid_nb = { 'var_smoothing': np.logspace(0,-9, num=100) }

nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
nbModel_grid.fit(X_train_pca, y_train)

print("Test set score: ", nbModel_grid.score(X_test_pca, y_test))
print("Best parameters: {}".format(nbModel_grid.best_params_))
print("Best cross-validation score: ", nbModel_grid.best_score_)

ideal_var_smoothing = nbModel_grid.best_params_['var_smoothing']
nb = GaussianNB(var_smoothing=ideal_var_smoothing)
nb.fit(X_train_pca, y_train)
y_pred = nb.predict(X_test_pca)
print("\nAccuracy on test set after parameter tuning: {:.3f}".format(accuracy_score(y_test, y_pred)))

Decision Trees

In [440]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)
tree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)

In [490]:
for depth in range(1,20):
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    print('Depth:' + str(depth) + ' ; Accuracy:' + str(tree.score(X_test, y_test)))

Depth:1 ; Accuracy:0.20066666666666666
Depth:2 ; Accuracy:0.3564
Depth:3 ; Accuracy:0.48133333333333334
Depth:4 ; Accuracy:0.5986666666666667
Depth:5 ; Accuracy:0.648
Depth:6 ; Accuracy:0.6956
Depth:7 ; Accuracy:0.734
Depth:8 ; Accuracy:0.7557333333333334
Depth:9 ; Accuracy:0.7598666666666667
Depth:10 ; Accuracy:0.7769333333333334
Depth:11 ; Accuracy:0.7721333333333333
Depth:12 ; Accuracy:0.7678666666666667
Depth:13 ; Accuracy:0.7673333333333333
Depth:14 ; Accuracy:0.7641333333333333
Depth:15 ; Accuracy:0.7641333333333333
Depth:16 ; Accuracy:0.7578666666666667
Depth:17 ; Accuracy:0.7613333333333333
Depth:18 ; Accuracy:0.7592
Depth:19 ; Accuracy:0.7592


In [441]:
# print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
# print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on training set: 0.908
Accuracy on test set: 0.776


In [560]:
# Bagging
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=50,
    max_samples=5000, bootstrap=True, random_state=42)
bag_clf.fit(X_train_pca, y_train)
y_pred = bag_clf.predict(X_test_pca)

In [561]:
print("Bagging ensemble of decision trees - accuracy on test set: {}".format(accuracy_score(y_test, y_pred)))

Bagging ensemble of decision trees - accuracy on test set: 0.8185333333333333


In [564]:
# AdaBoost 
for n in [200,300,500,1000]:
    ada_clf = AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=1), n_estimators=n, learning_rate=0.5, random_state=42)
    ada_clf.fit(X_train_pca, y_train)
    y_pred_ada = ada_clf.predict(X_test_pca)
    print("AdaBoost ensemble - accuracy on test set:")
    print(accuracy_score(y_test, y_pred_ada))

AdaBoost ensemble - accuracy on test set:
0.566
AdaBoost ensemble - accuracy on test set:
0.5517333333333333


KeyboardInterrupt: 

SVM

In [494]:
# SVM with Linear Kernel
lin_svm = SVC(kernel="linear")
lin_svm.fit(X_train_pca, y_train)

y_pred = lin_svm.predict(X_test_pca)
print("Linear SVM - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Linear SVM - accuracy on test set: 0.855


In [495]:
# SVM with polynomial kernel
poly_svm = SVC(kernel="poly", degree=2)#polynomial kernel with degree 2
poly_svm.fit(X_train_pca, y_train)
y_pred = poly_svm.predict(X_test_pca)
print("SVM with polynomial kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

SVM with polynomial kernel - accuracy on test set: 0.877


In [496]:
# SVM with RBF kernel
rbf_svm = SVC(kernel="rbf", gamma="auto")
rbf_svm.fit(X_train_pca, y_train)
y_pred = rbf_svm.predict(X_test)
print("SVM with RBF kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

SVM with RBF kernel - accuracy on test set: 0.867


In [ ]:
param_grid = {'C': [0.0001], 'gamma': [0.01],'kernel': ['poly']}
svmModel_grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
svmModel_grid.fit(X_train_pca,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..................C=0.0001, gamma=0.01, kernel=poly; total time= 3.2min
[CV] END ..................C=0.0001, gamma=0.01, kernel=poly; total time= 3.2min
[CV] END ..................C=0.0001, gamma=0.01, kernel=poly; total time= 3.1min


In [107]:
y_pred = svmModel_grid.predict(X_test_pca)
print('Grid search SVM Accuracy: {}'.format(accuracy_score(y_test, y_pred)))

Grid search SVM Accuracy: 0.8865333333333333


In [108]:
svmModel_grid.score(X_train_pca, y_train)

0.9973333333333333

In [89]:
predictions = []

for i in output_test_pca:
    prediction = svmModel_grid.predict([list(i)])
    predictions.append(prediction[0])

predictions

[1,
 1,
 1,
 0,
 6,
 6,
 1,
 2,
 9,
 5,
 9,
 9,
 8,
 4,
 6,
 4,
 0,
 2,
 5,
 3,
 6,
 4,
 8,
 7,
 6,
 2,
 9,
 6,
 5,
 5,
 8,
 7,
 8,
 3,
 7,
 7,
 6,
 6,
 3,
 3,
 6,
 0,
 2,
 6,
 3,
 2,
 4,
 5,
 0,
 0,
 4,
 8,
 9,
 7,
 3,
 0,
 7,
 1,
 3,
 9,
 8,
 6,
 5,
 0,
 5,
 0,
 2,
 2,
 9,
 6,
 7,
 2,
 3,
 9,
 9,
 8,
 5,
 1,
 1,
 7,
 2,
 6,
 7,
 8,
 5,
 9,
 2,
 0,
 7,
 6,
 3,
 4,
 0,
 3,
 3,
 9,
 6,
 5,
 0,
 8,
 7,
 9,
 4,
 3,
 1,
 4,
 6,
 7,
 2,
 9,
 2,
 3,
 9,
 3,
 4,
 5,
 6,
 7,
 8,
 3,
 6,
 2,
 3,
 7,
 9,
 8,
 6,
 0,
 3,
 0,
 4,
 1,
 5,
 6,
 4,
 6,
 7,
 7,
 3,
 8,
 8,
 8,
 2,
 1,
 8,
 1,
 9,
 2,
 1,
 3,
 3,
 4,
 4,
 2,
 8,
 8,
 6,
 0,
 1,
 2,
 3,
 0,
 9,
 2,
 4,
 6,
 2,
 7,
 7,
 4,
 8,
 7,
 3,
 1,
 2,
 1,
 3,
 8,
 7,
 2,
 7,
 4,
 6,
 5,
 9,
 0,
 6,
 6,
 5,
 9,
 2,
 8,
 9,
 2,
 0,
 1,
 7,
 9,
 0,
 9,
 7,
 5,
 6,
 7,
 7,
 4,
 9,
 7,
 0,
 2,
 4,
 5,
 7,
 2,
 4,
 7,
 1,
 5,
 0,
 5,
 9,
 6,
 6,
 0,
 1,
 5,
 1,
 5,
 7,
 7,
 9,
 5,
 7,
 7,
 6,
 6,
 9,
 8,
 3,
 6,
 0,
 7,
 9,
 3,
 0,
 8,
 6,
 3,
 1,
 3,


In [90]:
output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('test_output_svm_poly.csv', sep=",", float_format='%d', index_label="id")